# Smoke Test for LLM Personality Pipeline

步骤：
1. 设定环境变量 `OPENAI_API_KEY` 或 `OPENROUTER_API_KEY`（以及 `OPENROUTER_BASE_URL`）。
2. 选择 1 个模型 + 1 个 persona 进行快速验证。
3. 生成故事与行为数据，查看输出路径。


In [ ]:
from pathlib import Path
from src.story_gen import async_generate_story, load_personas, load_writing_prompt
from src.behavior_gen import (
    async_run_bfi_task,
    async_run_risk_task,
    async_run_iat_task,
    async_run_honesty_task,
    async_run_sycophancy_task,
    load_bfi_template,
    load_iat_stimuli,
    load_dilemmas,
)
import asyncio, json

# 配置
MODEL = "gpt-4o-mini"  # 可替换
PERSONA_INDEX = 0  # 取 personas.json 中的第一个
stories_dir = Path("data/outputs/stories")
behaviors_dir = Path("data/outputs/behaviors")
stories_dir.mkdir(parents=True, exist_ok=True)
behaviors_dir.mkdir(parents=True, exist_ok=True)

personas = load_personas(Path("data/inputs/personas.json"))
persona = personas[PERSONA_INDEX]
writing_template = load_writing_prompt(Path("data/inputs/writing_prompt.txt"))
bfi_template = load_bfi_template(Path("data/inputs/bfi_prompt.txt"))
iat_data = load_iat_stimuli()
dilemmas = load_dilemmas()

async def run_smoke():
    story_path = await async_generate_story(
        model=MODEL,
        persona=persona,
        writing_template=writing_template,
        output_dir=stories_dir,
    )
    bfi = await async_run_bfi_task(MODEL, persona, bfi_template)
    risk = await async_run_risk_task(MODEL, persona)
    iat = await async_run_iat_task(MODEL, persona, stimuli=iat_data)
    honesty = await async_run_honesty_task(MODEL, persona)
    syc = await async_run_sycophancy_task(MODEL, persona, dilemmas=dilemmas)
    payload = {
        "meta": {"model": MODEL, "persona_id": persona["id"], "traits": persona["traits"]},
        "behaviors": {"bfi": bfi, "risk": risk, "iat": iat, "honesty": honesty, "sycophancy": syc},
    }
    out = behaviors_dir / f"{MODEL.replace('/', '_')}_{persona['id']}_smoke.json"
    out.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")
    print("Story saved to", story_path)
    print("Behaviors saved to", out)

await run_smoke()

> 如需仅跑故事或单个行为任务，可注释对应代码。默认并发为单线程，便于在 notebook 里查看输出。